In [1]:
import pandas as pd

df_confirmed = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv', index_col='Country/Region')
df_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv', index_col='Country/Region')
df_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv', index_col='Country/Region')

In [2]:
df_confirmed.rename(columns={'Province/State':'Province'}, inplace=True)
df_deaths.rename(columns={'Province/State':'Province'}, inplace=True)
df_recovered.rename(columns={'Province/State':'Province'}, inplace=True)

In [3]:
grpC = df_confirmed.groupby(['Country/Region'])
grpD = df_deaths.groupby(['Country/Region'])
grpR = df_recovered.groupby(['Country/Region'])

In [16]:
for (colC, colD, colR) in zip(df_confirmed.columns[3:], df_deaths.columns[3:], df_recovered.columns[3:]):

    Confirmed = df_confirmed[['Province', colC]]
    Deaths = df_deaths[['Province', colD]]
    Recovered = df_recovered[['Province', colR]]

    filt = pd.isnull(Confirmed.Province)
    filt1 = pd.isnull(Deaths.Province)
    filt2 = pd.isnull(Recovered.Province)

    Confirmed = Confirmed.loc[filt]
    Deaths = Deaths.loc[filt1]
    Recovered = Recovered.loc[filt2]
    
    Confirmed.rename(columns={colC:'Confirmed_Cases'}, inplace=True)
    Deaths.rename(columns={colD:'Total_Deaths'}, inplace=True)
    Recovered.rename(columns={colR:'Total_Recovered'}, inplace=True)
    
    new = pd.concat([Confirmed, Deaths, Recovered], axis='columns')

    new.drop(['Province'], axis=1, inplace=True)

    chinaC = grpC.get_group('China')[colC].sum()
    canadaC = grpC.get_group('Canada')[colC].sum()
    australiaC = grpC.get_group('Australia')[colC].sum()

    chinaD = grpD.get_group('China')[colD].sum()
    canadaD = grpD.get_group('Canada')[colD].sum()
    australiaD = grpD.get_group('Australia')[colD].sum()

    australiaR = grpR.get_group('Australia')[colR].sum()
    canadaR = Recovered.loc['Canada', 'Total_Recovered']
    chinaR = grpR.get_group('China')[colR].sum()

    Arow = pd.Series(data={'Confirmed_Cases': australiaC, 'Total_Deaths': australiaD,
                           'Total_Recovered': australiaR}, name='Australia')
    new = new.append(Arow, ignore_index=False)

    Chrow = pd.Series(data={'Confirmed_Cases': chinaC,
                            'Total_Deaths': chinaD, 'Total_Recovered': chinaR}, name='China')
    new = new.append(Chrow, ignore_index=False)

    new.drop(['Canada'], axis=0, inplace=True)

    Carow = pd.Series(data={'Confirmed_Cases': canadaC,
                            'Total_Deaths': canadaD, 'Total_Recovered': canadaR}, name='Canada')
    new = new.append(Carow, ignore_index=False)
    
    new['Active_Cases'] = new['Confirmed_Cases'] - new['Total_Deaths'] - new['Total_Recovered']
    
    new = new.astype(int)
    temp = colC.replace('/', '-')
    new.to_json(f'COVID-19\\JHU_{temp}.json', indent=2)
    